In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from random import seed
from random import randrange
import seaborn as sns

In [2]:
# Lectura de datos
data = sns.load_dataset('iris')
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
# Cambiar ajuste de datos para clasificar setosa
data['categoria'] = [1 if x =='setosa' else -1 for x in data['species']] 

In [4]:
# Función clasifica
def clasifica(X, Y, theta, theta_0):
    
    """ Funcion que realiza la evaluacion de la prediccion contra las eitquetas:
    theta - parametro theta
    theta_0 - parametro theta0
    X - vector de características a clasificar
    Y - vector de etiquetas de las características (-1 o 1)    

    Outputs:
    value = -1 en caso de tener una clasificacion erronea. 0 eoc
    """
    
    if Y*(np.dot(theta,np.transpose(X)) + theta_0) < 1 :
        value = -1
    else:
        value = 0

    return value

In [5]:
def loess(X, Y, theta, theta_0):
    """ Funcion que realiza la evaluacion en en la funcion de perdida:
    X - vector de características a clasificar
    Y - vector de etiquetas de las características (-1 o 1)        
    theta - parametro theta
    theta_0 - parametro theta0

    Outputs:
    value= valor de perdida
     """
    
    suma = 0
    H = 0
    n = X.shape[0]
    
    for i in range(n):
        H = Y[i]*(np.dot(theta, X[i]) + theta_0)
        if H >= 1:
            suma += 0
        else:
            suma += (1-H)
            
    return suma

In [6]:
def jacob(X, Y, theta, theta_0, lamb):
    """ Funcion que realiza la evaluacionen de la clasiificaciòn lineal con margen como optimizacion
    X - vector de características a clasificar
    Y - vector de etiquetas de las características (-1 o 1)        
    theta- parametro theta
    theta0 - parametro theta0
    lamb - parametro de regularizacion

    Outputs:
    value = valor del jacobiano
     """
    
    n = X.shape[0]
    jac = loess(X, Y, theta, theta_0)/n + (lamb/2.0)*np.linalg.norm(theta)

    return jac

In [14]:
# Función para descenso por gradiente
def SVM(X, Y, theta, theta_0, etha, lamb, eps = 1e-8, MAX = 5000):
    """ Funcion que realiza el aglrotimo de Support Vector Machine - Descenso en Gradiente
    Inputs:
    X - vector de características a clasificar
    Y - vector de etiquetas de las características (-1 o 1)    
    theta - incializacion de parametro theta
    theta_0 - inicializacion de parametro theta0
    etha - tasa de aprendizaje
    lamb - parametro de regularizacion
    
    Outputs:
    tetha = vector para formar el plano
    tetha_0 = escalar para desplazar el plano 
    """
    
    n = X.shape[0]
    sum_theta = 0
    sum_theta_0 = 0
    error = 10
    t = 0
    
    while error >= eps and t < MAX:
        # Guardar parametros en tiempo: t-1
        theta_old = theta
        theta_0_old = theta_0
        
        # Evaluacion de la suma para la actualizacion el descenso por gradiente
        for i in range(n):
            sum_theta += clasifica(X[i], Y[i], theta, theta_0)*Y[i]*X[i]
            sum_theta_0 += clasifica(X[i], Y[i], theta, theta_0)*Y[i]
        
        # Descenso por gradiente
        theta = theta - etha*((sum_theta/n) + lamb*theta)
        theta_0= theta_0 - etha*((sum_theta_0/n))
        
        # Calculo del error
        error = abs(jacob(X, Y, theta, theta_0, lamb) - jacob(X, Y, theta_old, theta_0_old, lamb))
        t += 1
    
    jac = jacob(X, Y, theta, theta_0, lamb)
    return theta, theta_0, t, error, jac

In [47]:
def error(X, Y, theta, theta_0):
    """ Funcion que calcula el error
    
    Inputs:
    X - vector de caracteristicas numericas
    y - vector de etiquetas de las características (-1 o 1)
    Nota: se asume que X y y tienen la misma cantidad de registros
    
    theta = vector para formar el plano
    theta_0 = escalar para desplazar el plano (sesgo)
    
    Outputs:
    ee - error promedio de las observaciones que son mal clasificadas
    
    """
    
    n = len(X)
    suma = 0
    ee = 0
    
    for i in np.arange(n):
        clasificador = Y[i]*((np.dot(theta,x[i])) + theta_0)
        if clasificador <= 0:
            suma += 1         
    ee += suma/n
    return ee

In [119]:
def SGD(X, Y, lamb, eps = 1e-8, T = 5000):
    """ Funcion que los parámetros del modelo perceptron 
    Inputs:
    X_array - arreglo de caracteristicas numericas
    y_array - arreglo etiquetas de las características (-1 o 1)
    Nota: se asume que X y y tienen la misma cantidad de registros
    
    T -  Número de iteraciones
    lambda_ - Párametro lambda 
    
    Outputs:
    theta_t - parámetro theta e intercepto (theta_t[-1])
    
    """
    
    #n = X.shape[0]
    #ones = np.matrix(np.ones((X.shape[0], 1)))
    #X = np.append(X, ones, axis=1)
    #Y = np.matrix(Y).reshape(Y.shape[0],1)
    t = 0
    #error = 10
    #theta_t=np.matrix(np.zeros(X.shape[1]))
    
    
    n = X.shape[0]
    X = np.matrix(X)
    ones = np.matrix(np.ones((X.shape[0],1)))
    X = np.append(X, ones, axis=1)
    Y = np.matrix(Y).reshape(Y.shape[0],1)
    theta_t = np.matrix(np.zeros(X.shape[1]))
    
    #while error >= eps and t <= T:
    while t <= T:
        # Guardar parametros en tiempo: t-1
        theta_old = theta
        
        i = np.random.randint(n)
        etha = 1/(t+1)
        theta_t = np.squeeze(np.asarray(theta_t))
        x_i = np.squeeze(np.asarray(X[i]))
        theta_t = theta_t - etha*((np.dot(theta_t, x_i) - Y[i])*x_i + (lamb/n)*theta_t)
        t += 1
        
        # Calculo del error
        # error = abs(jacob(X, Y, theta, theta_0, lamb) - jacob(X, Y, theta_old, theta_0_old, lamb))
        
    return np.array(theta_t), t

In [120]:
def CV_SGD(D, k, lamb):
    """ Funcion que realiza la validación cruzada para determinar con que hiperprámetros se minimiza el error
    Inputs:
    df - Data frame del conjunto de entrenamiento
    k - El número de subcojunto de datos en el cual se desea dividir el set de train

    Outputs:
    resultados - Un data frame que arroja los errores para cada k 
    """
    
    error = []
    n = D.shape[0]
    n = int((n / k))
    
    for i in range (0,k):
        init = n*i
        fin = n*(1+i)
        test_i = D.iloc[init:fin,:]
        train_i = D.drop(test_i.index)
        
        # Calcular theta y theta_0 con SGD y train
        X = train_i.iloc[:,0:4]
        Y = train_i.iloc[:,5]
        theta_t = SGD(X = X, Y = Y, lamb = lamb)
        theta = theta_t[0,0:-1]
        theta_0 = theta_t[0,-1]
        
        #Se calcula el error en esa primer corrida
        X = test_i.iloc[:,0:4]   #Columna con las 4 características
        Y = testi_i.iloc[:,5]    #Columna con el vector de etiquetas
        e_i = error(X = X, Y = Y, theta = theta, theta_0 = theta_0)
        error.append(e_i)
    
    error_promedio = np.mean(error)

    return error_promedio

In [121]:
# Separar datos para prueba y entrenamiento
train = data.sample(frac = 0.8, random_state = 2020) # Fijamos la semilla con random_state
test = data.drop(train.index)

# Extraer variables para la clasificación
data_vars = train.iloc[:,0:4]
vars_train = data_vars.to_numpy()
data_labels = train.iloc[:,5]
labels_train = data_labels.to_numpy()

theta = np.array([0.0, 0.0, 0.0, 0.0])
theta_0 = 0.0
etha = 0.0001
lamb = 0.0001

In [122]:
# Probar SGD
error_sgd = []
lambdas = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.1]
for lamb in lambdas:
    error_sgd.append(CV_SGD(D = train, k = 5, lamb = lamb))

print(errors_sgd)

TypeError: tuple indices must be integers or slices, not tuple

In [97]:
theta_opt, theta_0_opt, t, error, jac = SVM(X = vars_train, Y = labels_train, theta = theta, 
                                            theta_0 = theta_0, etha = etha, lamb = lamb)

In [27]:
print('Num iteraciones: ', t)
print('Error de aprox: ', error)
print('Jacobiano de aprox: ', jac)

Num iteraciones:  5000
Error de aprox:  7.052974922466988e-07
Jacobiano de aprox:  0.0034621843893957614


In [46]:
theta_sgd,t = SGD(X = vars_train, Y = labels_train, lamb = lamb, eps = 1e-8, T = 5000)

In [45]:
print(theta_sgd)
print(t)

[[ 1.36836456e+15 -1.70663671e+15 -6.23471558e+14 -1.75923630e+14]]
501


In [79]:
i = 0
D = train
n = D.shape[0]
k = 5
n = int((n / k))

In [98]:
init = n*i
fin = n*(1+i)
test_i = D.iloc[init:fin,:]
train_i = D.drop(test_i.index)

# Calcular theta y theta_0 con SGD y train
X = train_i.iloc[:,0:4]
Y = train_i.iloc[:,5]
theta_t = SGD(X = X, Y = Y, lamb = lamb)
theta = theta_t[0,0:-1]
theta_0 = theta_t[0,-1]

ValueError: shapes (1,5) and (1,5) not aligned: 5 (dim 1) != 1 (dim 0)

In [99]:
SGD(X = X, Y = Y, lamb = lamb)

ValueError: shapes (1,5) and (1,5) not aligned: 5 (dim 1) != 1 (dim 0)

In [100]:
n = X.shape[0]
X = np.matrix(X)
ones = np.matrix(np.ones((X.shape[0],1)))
X = np.append(X, ones, axis=1)
Y = np.matrix(Y).reshape(Y.shape[0],1)
theta_t = np.matrix(np.zeros(X.shape[1]))

In [101]:
theta_t = theta_t - etha*((np.dot(theta_t, X[i]) - Y[i])*X[i] + (lamb/n)*theta_t)

ValueError: shapes (1,5) and (1,5) not aligned: 5 (dim 1) != 1 (dim 0)

In [118]:
theta_t_1 = np.squeeze(np.asarray(theta_t))
X_1 = np.squeeze(np.asarray(X[i]))

np.dot(theta_t_1, X_1)

0.0

In [115]:
theta_tX[i]

matrix([[0., 0., 0., 0., 0.]])

In [116]:
X[i][0]

matrix([[5.8, 4. , 1.2, 0.2, 1. ]])

In [112]:
X[i]%*%theta_t

SyntaxError: invalid syntax (<ipython-input-112-b40c7c4cb3b9>, line 1)